https://adventofcode.com/2018/day/12

In [1]:
# part 1
function play1(state, notes, N; padding = 100)
    P = repeat(".", padding)
    S = P * state * P
    for i in 1:N
        V = fill("", length(S)-4)   #less 4
        for i in 1:length(S)-4
            k = S[i:i+4]
            V[i] = k ∈ keys(notes) ? notes[k] : "."
        end
        S = ".." * join(V) * ".."
#         println(S)
    end
    L = length(state)
    Q = collect(-padding:L+padding-1)
    B = [x == '#' for x in S]
#     return (S, Q[B], sum(Q[B]))
    sum(Q[B])
end

# sample
state_txt = "#..#.#..##......###...###"
notes_txt = """
...## => #
..#.. => #
.#... => #
.#.#. => #
.#.## => #
.##.. => #
.#### => #
#.#.# => #
#.### => #
##.#. => #
##.## => #
###.. => #
###.# => #
####. => #"""
notes = Dict(x[1] => x[2] for x in split.(split(notes_txt, "\n"), " => "))
@time @show play1(state_txt, notes, 20)

state_txt = "##.####..####...#.####..##.#..##..#####.##.#..#...#.###.###....####.###...##..#...##.#.#...##.##.."
notes_txt = """
##.## => #
....# => .
.#.#. => #
..### => .
##... => #
##### => .
###.# => #
.##.. => .
..##. => .
...## => #
####. => .
###.. => .
.#### => #
#...# => #
..... => .
..#.. => .
#..## => .
#.#.# => #
.#.## => #
.###. => .
##..# => .
.#... => #
.#..# => #
...#. => .
#.#.. => .
#.... => .
##.#. => .
#.### => .
.##.# => .
#..#. => #
..#.# => .
#.##. => #"""
notes = Dict(x[1] => x[2] for x in split.(split(notes_txt, "\n"), " => "))
@time @show play1(state_txt, notes, 20)
# @time @show play(state_txt, notes, 100)

# BUG - not working with higher number of generations...
@time @show play1(state_txt, notes, 200)
@time @show play1(state_txt, notes, 300)

# @time for i in 100:100:2000
#     println(play(state_txt, notes, i))
# end
;

play1(state_txt, notes, 20) = 325
  0.389085 seconds (511.07 k allocations: 25.199 MiB, 2.85% gc time)
play1(state_txt, notes, 20) = 1623
  0.004019 seconds (39.62 k allocations: 1.695 MiB)
play1(state_txt, notes, 200) = 2091
  0.037164 seconds (405.79 k allocations: 17.449 MiB, 28.07% gc time)
play1(state_txt, notes, 300) = 0
  0.052876 seconds (586.68 k allocations: 25.220 MiB, 11.49% gc time)


In [2]:
const empty_bool_array = Bool[]

function ensure_padding(state::Vector{Bool}, result::Vector{Bool}, origin::Int)
    padding = 5
    
    true1 = findmax(state)[2]
    add1 = padding - true1 + 1
    L = add1 > 0 ? [false for _ in 1:add1] : empty_bool_array
    origin = add1 > 0 ? origin - add1 : origin

    true2 = findmax(reverse(state))[2]
    add2 = padding - true2 + 1
    R = add2 > 0 ? [false for _ in 1:add2] : empty_bool_array

    if add1 > 0 || add2 > 0
        new_state = [L; state ; R]
        new_result = [false for _ in 1:length(new_state)]
#         visualize(new_state)
        return new_state, new_result, origin
    else
        return state, result, origin
    end
end

function generate!(state::Vector{Bool}, result::Vector{Bool}, notes::Dict{Vector{Bool},Bool})
    for i in 1:length(state)-4
        k = state[i:i+4]
        result[i+2] = k ∈ keys(notes) ? notes[k] : false
    end
end

function visualize(state::Vector{Bool})
    println(join([b ? "#" : "." for b in state]))
end

function play2(state::Vector{Bool}, notes::Dict{Vector{Bool},Bool}, iterations::Int)
    result = Bool[false for _ in 1:length(state)]
    origin = 0
#     visualize(state)
    for i in 1:iterations
        state, result, origin = ensure_padding(state, result, origin)
        generate!(state, result, notes)
        state[:] = result[:]
#         visualize(state)
    end
    pot_range = collect(0:length(state)-1) .+ origin
    return pot_range[state], sum(pot_range[state])
end

play2 (generic function with 1 method)

In [3]:
state_txt = "##.####..####...#.####..##.#..##..#####.##.#..#...#.###.###....####.###...##..#...##.#.#...##.##.."
notes_txt = """
##.## => #
....# => .
.#.#. => #
..### => .
##... => #
##### => .
###.# => #
.##.. => .
..##. => .
...## => #
####. => .
###.. => .
.#### => #
#...# => #
..... => .
..#.. => .
#..## => .
#.#.# => #
.#.## => #
.###. => .
##..# => .
.#... => #
.#..# => #
...#. => .
#.#.. => .
#.... => .
##.#. => .
#.### => .
.##.# => .
#..#. => #
..#.# => .
#.##. => #"""
parse_pattern(s) = [x == '#' for x in s]
parse_result(s) = s == "#"
state = parse_pattern(state_txt)
notes = Dict(parse_pattern(x[1]) => parse_result(x[2]) for x in split.(split(notes_txt, "\n"), " => "))
@show play2(state, notes, 20)

# look for patterns
for i in 100:100:2000
    r = play2(state, notes, i)
    println("i = ", i, ", sum = ", r[2])
end

play2(state, notes, 20) = ([6, 7, 10, 11, 13, 17, 24, 28, 35, 36, 38, 41, 44, 45, 48, 50, 53, 57, 64, 68, 75, 82, 83, 86, 88, 93, 98, 103, 105, 115], 1623)
i = 100, sum = 3592
i = 200, sum = 6801
i = 300, sum = 10001
i = 400, sum = 13201
i = 500, sum = 16401
i = 600, sum = 19601
i = 700, sum = 22801
i = 800, sum = 26001
i = 900, sum = 29201
i = 1000, sum = 32401
i = 1100, sum = 35601
i = 1200, sum = 38801
i = 1300, sum = 42001
i = 1400, sum = 45201
i = 1500, sum = 48401
i = 1600, sum = 51601
i = 1700, sum = 54801
i = 1800, sum = 58001
i = 1900, sum = 61201
i = 2000, sum = 64401


In [4]:
# it seems to be linear.... calculate slope!
(64401 - 61201) / (2000 - 1900)

32.0

In [5]:
# y = 32x + b
# solve: b = y - 32x
64401 - 32 * 2000

401

In [6]:
# check
32 * 1600 + 401

51601

In [7]:
32 * 50_000_000_000 + 401

1600000000401